# BiteMe | Exploratory Data Analysis

In this notebook we analyse our images to help inform modelling.

In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

import cv2
import albumentations as A


ModuleNotFoundError: No module named 'albumentations'

In [17]:
# Define directories
base_dir_path = "../"

data_dir_path = os.path.join(base_dir_path, "data")
data_raw_dir_path = os.path.join(data_dir_path, "raw")
data_processed_dir_path = os.path.join(data_dir_path, "processed")

data_dir = os.listdir(data_dir_path)
data_raw_dir = os.listdir(data_raw_dir_path)

metadata_path = os.path.join(data_raw_dir_path, "metadata.csv")
metadata = pd.read_csv(metadata_path)

metadata.head()

,img_name,img_path,label
0,7059b14d2aa03ed6c4de11afa32591995181d31c.jpg,../data/raw/none/7059b14d2aa03ed6c4de11afa3259...,none
1,ea1b100b581fcdb7ddfae52cc62347a99e304ba4.jpg,../data/raw/none/ea1b100b581fcdb7ddfae52cc6234...,none
2,1a1442990ff143b7560e5757d9f76d37ab007f48.jpg,../data/raw/none/1a1442990ff143b7560e5757d9f76...,none
3,6eac051b9c45ff6821ec8675216f371711b7cea9.jpg,../data/raw/none/6eac051b9c45ff6821ec8675216f3...,none
4,fc72767f8520df9b2b83941077dc0ee013eb9399.jpg,../data/raw/none/fc72767f8520df9b2b83941077dc0...,none


In [18]:
metadata.value_counts("label")

label
tick        29
none        28
mosquito    28
horsefly    28
bedbug      28
ant         26
bee         25
mite        23
dtype: int64

In [19]:
print(f"Number of images: {len(metadata)}")

Number of images: 215


In [65]:
# Set image default dimensions
ROWS = 256
COLS = 256
CHANNELS = 3

In [107]:
def read_images(metadata_df, rows, cols, channels) -> np.array:
    """
    Takes in metadata dataframe and reads images to np.array
    
    Parameters
    ----------
    metadata_df : pd.Dataframe
        Metadata dataframe containing file paths.
        
    Returns
    -------
    np.array
        Multi-level array containing all images.    
    """
    
    img_list = []
    for img_path in metadata_df["img_path"]:
        # Read image in and resize
        img = cv2.imread(img_path)
        img = cv2.resize(img, (ROWS, COLS))
        # Append to list
        img_list.append(img)
        
    return np.array(img_list)

In [112]:
img_array = read_images(metadata, ROWS, COLS, CHANNELS)
print(f"Image array shape: {img_array.shape}")

Image array shape: (215, 256, 256, 3)


## Example Augmentations